In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 8)

import collections

import warnings
# warnings.filterwarnings('ignore')
from matplotlib.backends.backend_pdf import PdfPages

from pathlib import Path

import db_queries as db
import vivarium_helpers.id_helper as idh
import gbd_mapping
from vivarium import Artifact

# Add the repo directory vivarium_research_ciff_sam/ to sys.path
import os, sys
repo_path = os.path.abspath('../..')
sys.path.append(repo_path)
# Assumes vivarium_research_ciff_sam/ is in sys.path
import model_validation.vivarium_transformed_output as vto
# import model_validation.vivarium_raw_output as vro
import model_validation.vivarium_output_processing as vop
import model_validation.ciff_sam_results as csr

!pwd
!whoami
!date

/ihme/homes/ndbs/vivarium_research_ciff_sam/model_validation/model2
ndbs
Mon Aug 30 13:59:21 PDT 2021


In [2]:
%load_ext autoreload
%autoreload 2

In [16]:
data25 = csr.VivariumMeasures.cleaned_from_model_spec(2.5)
data25.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count']

In [17]:
data25.person_time = csr.get_total_person_time(data25)
print(data25.table_names())
data25.person_time

['wasting_transition_count', 'wasting_state_person_time', 'deaths', 'population', 'ylls', 'ylds', 'disease_state_person_time', 'disease_transition_count', 'person_time']


,age,input_draw,measure,scenario,sex,year,value
0,1-5_months,29,person_time,baseline,female,2022,44140.585900
1,1-5_months,29,person_time,baseline,female,2023,43843.549624
2,1-5_months,29,person_time,baseline,female,2024,43946.220397
3,1-5_months,29,person_time,baseline,female,2025,43707.561944
...,...,...,...,...,...,...,...
716,late_neonatal,946,person_time,baseline,male,2023,6263.611225
717,late_neonatal,946,person_time,baseline,male,2024,6284.217659
718,late_neonatal,946,person_time,baseline,male,2025,6291.129363
719,late_neonatal,946,person_time,baseline,male,2026,6238.913073


In [18]:
data25.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count',
 'person_time']

In [19]:
data25.wasting_state_person_time

,sex,year,wasting_state,measure,input_draw,scenario,value,age
0,female,2022,mild_child_wasting,state_person_time,29,baseline,414.090349,early_neonatal
1,female,2022,mild_child_wasting,state_person_time,29,baseline,1234.535250,late_neonatal
2,female,2022,mild_child_wasting,state_person_time,29,baseline,8926.954141,1-5_months
3,female,2022,mild_child_wasting,state_person_time,29,baseline,10434.387406,6-11_months
...,...,...,...,...,...,...,...,...
2876,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,30258.041068,1-5_months
2877,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,35695.241615,6-11_months
2878,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,70879.949350,12_to_23_months
2879,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,215444.187543,2_to_4


In [20]:
sam_person_time = data25.wasting_state_person_time.query(
    "wasting_state == 'severe_acute_malnutrition'"
)
sam_person_time

,sex,year,wasting_state,measure,input_draw,scenario,value,age
144,female,2022,severe_acute_malnutrition,state_person_time,29,baseline,43.431896,early_neonatal
145,female,2022,severe_acute_malnutrition,state_person_time,29,baseline,125.164956,late_neonatal
146,female,2022,severe_acute_malnutrition,state_person_time,29,baseline,923.686516,1-5_months
147,female,2022,severe_acute_malnutrition,state_person_time,29,baseline,1079.883641,6-11_months
...,...,...,...,...,...,...,...,...
2804,male,2026,severe_acute_malnutrition,state_person_time,946,baseline,1795.195072,1-5_months
2805,male,2026,severe_acute_malnutrition,state_person_time,946,baseline,2040.405202,6-11_months
2806,male,2026,severe_acute_malnutrition,state_person_time,946,baseline,3063.549624,12_to_23_months
2807,male,2026,severe_acute_malnutrition,state_person_time,946,baseline,5503.170431,2_to_4


In [22]:
data25.wasting_transition_count.transition.unique()

array(['mild_child_wasting_to_moderate_acute_malnutrition',
       'mild_child_wasting_to_susceptible_to_child_wasting',
       'moderate_acute_malnutrition_to_mild_child_wasting',
       'moderate_acute_malnutrition_to_severe_acute_malnutrition',
       'severe_acute_malnutrition_to_mild_child_wasting',
       'severe_acute_malnutrition_to_moderate_acute_malnutrition',
       'susceptible_to_child_wasting_to_mild_child_wasting'], dtype=object)

In [23]:
data25.wasting_transition_count.query(
    "transition == 'moderate_acute_malnutrition_to_severe_acute_malnutrition'"
)

,sex,year,measure,input_draw,scenario,value,age,transition
216,female,2022,transition_count,29,baseline,274.0,early_neonatal,moderate_acute_malnutrition_to_severe_acute_ma...
217,female,2022,transition_count,29,baseline,736.0,late_neonatal,moderate_acute_malnutrition_to_severe_acute_ma...
218,female,2022,transition_count,29,baseline,6420.0,1-5_months,moderate_acute_malnutrition_to_severe_acute_ma...
219,female,2022,transition_count,29,baseline,7454.0,6-11_months,moderate_acute_malnutrition_to_severe_acute_ma...
...,...,...,...,...,...,...,...,...
4820,male,2026,transition_count,946,baseline,12719.0,1-5_months,moderate_acute_malnutrition_to_severe_acute_ma...
4821,male,2026,transition_count,946,baseline,13308.0,6-11_months,moderate_acute_malnutrition_to_severe_acute_ma...
4822,male,2026,transition_count,946,baseline,19839.0,12_to_23_months,moderate_acute_malnutrition_to_severe_acute_ma...
4823,male,2026,transition_count,946,baseline,36682.0,2_to_4,moderate_acute_malnutrition_to_severe_acute_ma...


In [25]:
sam_duration = csr.get_sam_duration(data25)
sam_duration

,year,sex,age,input_draw,scenario,value,numerator_measure,denominator_measure,multiplier
0,2022,female,1-5_months,29,baseline,0.143876,state_person_time,transition_count,1
1,2022,female,1-5_months,223,baseline,0.146593,state_person_time,transition_count,1
2,2022,female,1-5_months,232,baseline,0.146573,state_person_time,transition_count,1
3,2022,female,1-5_months,357,baseline,0.135437,state_person_time,transition_count,1
...,...,...,...,...,...,...,...,...,...
716,2026,male,late_neonatal,650,baseline,0.172432,state_person_time,transition_count,1
717,2026,male,late_neonatal,680,baseline,0.150054,state_person_time,transition_count,1
718,2026,male,late_neonatal,829,baseline,0.159562,state_person_time,transition_count,1
719,2026,male,late_neonatal,946,baseline,0.151926,state_person_time,transition_count,1


In [26]:
365 * vop.value(sam_duration)

value
age           denominator_measure input_draw multiplier numerator_measure scenario sex    year           
1-5_months    transition_count    29         1          state_person_time baseline female 2022  52.514888
                                  223        1          state_person_time baseline female 2022  53.506577
                                  232        1          state_person_time baseline female 2022  53.499115
                                  357        1          state_person_time baseline female 2022  49.434549
...                                                                                                   ...
late_neonatal transition_count    650        1          state_person_time baseline male   2026  62.937766
                                  680        1          state_person_time baseline male   2026  54.769566
                                  829        1          state_person_time baseline male   2026  58.240191
                                  946        1          state_person_time baseline male   2026  55.453053

[720 rows x 1 columns]

In [29]:
data25.compute_sam_duration()
data25.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count',
 'person_time',
 'sam_duration']

In [30]:
csr.get_mam_duration(data25)

,year,sex,age,input_draw,scenario,value,numerator_measure,denominator_measure,multiplier
0,2022,female,1-5_months,29,baseline,0.110263,state_person_time,transition_count,1
1,2022,female,1-5_months,223,baseline,0.105621,state_person_time,transition_count,1
2,2022,female,1-5_months,232,baseline,0.109135,state_person_time,transition_count,1
3,2022,female,1-5_months,357,baseline,0.106226,state_person_time,transition_count,1
...,...,...,...,...,...,...,...,...,...
716,2026,male,late_neonatal,650,baseline,0.104895,state_person_time,transition_count,1
717,2026,male,late_neonatal,680,baseline,0.097768,state_person_time,transition_count,1
718,2026,male,late_neonatal,829,baseline,0.099833,state_person_time,transition_count,1
719,2026,male,late_neonatal,946,baseline,0.099789,state_person_time,transition_count,1


In [32]:
vop.describe(365*vop.value(csr.get_mam_duration(data25)))

count  \
age           denominator_measure multiplier numerator_measure scenario sex    year          
1-5_months    transition_count    1          state_person_time baseline female 2022   12.0   
                                                                               2023   12.0   
                                                                               2024   12.0   
                                                                               2025   12.0   
...                                                                                    ...   
late_neonatal transition_count    1          state_person_time baseline male   2023   12.0   
                                                                               2024   12.0   
                                                                               2025   12.0   
                                                                               2026   12.0   

                                                                                          mean  \
age           denominator_measure multiplier numerator_measure scenario sex    year              
1-5_months    transition_count    1          state_person_time baseline female 2022  39.572052   
                                                                               2023  39.616303   
                                                                               2024  39.446049   
                                                                               2025  39.488411   
...                                                                                        ...   
late_neonatal transition_count    1          state_person_time baseline male   2023  36.918519   
                                                                               2024  36.713866   
                                                                               2025  36.818172   
                                                                               2026  36.673403   

                                                                                          std  \
age           denominator_measure multiplier numerator_measure scenario sex    year             
1-5_months    transition_count    1          state_person_time baseline female 2022  0.828871   
                                                                               2023  0.660100   
                                                                               2024  0.616794   
                                                                               2025  0.582509   
...                                                                                       ...   
late_neonatal transition_count    1          state_person_time baseline male   2023  0.994370   
                                                                               2024  0.781361   
                                                                               2025  0.740353   
                                                                               2026  0.803983   

                                                                                           min  \
age           denominator_measure multiplier numerator_measure scenario sex    year              
1-5_months    transition_count    1          state_person_time baseline female 2022  38.551617   
                                                                               2023  38.796504   
                                                                               2024  38.694611   
                                                                               2025  38.730358   
...                                                                                        ...   
late_neonatal transition_count    1          state_person_time baseline male   2023  35.129515   
                                                                               2024  35.498099   
                     

In [33]:
365/6.7

54.47761194029851

In [34]:
data25.compute_mam_duration()

In [35]:
data25.mam_duration

,year,sex,age,input_draw,scenario,value,numerator_measure,denominator_measure,multiplier
0,2022,female,1-5_months,29,baseline,0.110263,state_person_time,transition_count,1
1,2022,female,1-5_months,223,baseline,0.105621,state_person_time,transition_count,1
2,2022,female,1-5_months,232,baseline,0.109135,state_person_time,transition_count,1
3,2022,female,1-5_months,357,baseline,0.106226,state_person_time,transition_count,1
...,...,...,...,...,...,...,...,...,...
716,2026,male,late_neonatal,650,baseline,0.104895,state_person_time,transition_count,1
717,2026,male,late_neonatal,680,baseline,0.097768,state_person_time,transition_count,1
718,2026,male,late_neonatal,829,baseline,0.099833,state_person_time,transition_count,1
719,2026,male,late_neonatal,946,baseline,0.099789,state_person_time,transition_count,1


In [37]:
vop.describe(1/vop.value(data25.sam_duration))

count  \
age           denominator_measure multiplier numerator_measure scenario sex    year          
1-5_months    transition_count    1          state_person_time baseline female 2022   12.0   
                                                                               2023   12.0   
                                                                               2024   12.0   
                                                                               2025   12.0   
...                                                                                    ...   
late_neonatal transition_count    1          state_person_time baseline male   2023   12.0   
                                                                               2024   12.0   
                                                                               2025   12.0   
                                                                               2026   12.0   

                                                                                         mean  \
age           denominator_measure multiplier numerator_measure scenario sex    year             
1-5_months    transition_count    1          state_person_time baseline female 2022  6.759653   
                                                                               2023  6.813434   
                                                                               2024  6.803735   
                                                                               2025  6.798936   
...                                                                                       ...   
late_neonatal transition_count    1          state_person_time baseline male   2023  6.512297   
                                                                               2024  6.584634   
                                                                               2025  6.469740   
                                                                               2026  6.571454   

                                                                                          std  \
age           denominator_measure multiplier numerator_measure scenario sex    year             
1-5_months    transition_count    1          state_person_time baseline female 2022  0.280768   
                                                                               2023  0.244544   
                                                                               2024  0.242004   
                                                                               2025  0.227243   
...                                                                                       ...   
late_neonatal transition_count    1          state_person_time baseline male   2023  0.388001   
                                                                               2024  0.399989   
                                                                               2025  0.506949   
                                                                               2026  0.487839   

                                                                                          min  \
age           denominator_measure multiplier numerator_measure scenario sex    year             
1-5_months    transition_count    1          state_person_time baseline female 2022  6.335826   
                                                                               2023  6.444981   
                                                                               2024  6.473028   
                                                                               2025  6.393084   
...                                                                                       ...   
late_neonatal transition_count    1          state_person_time baseline male   2023  5.653460   
                                                                               2024  5.675006   
                                         

In [39]:
data25.compute_mam_duration('age')
data25.mam_duration

,age,input_draw,scenario,value,numerator_measure,denominator_measure,multiplier
0,1-5_months,29,baseline,0.104402,state_person_time,transition_count,1
1,1-5_months,223,baseline,0.102696,state_person_time,transition_count,1
2,1-5_months,232,baseline,0.105386,state_person_time,transition_count,1
3,1-5_months,357,baseline,0.103154,state_person_time,transition_count,1
...,...,...,...,...,...,...,...
68,late_neonatal,650,baseline,0.106789,state_person_time,transition_count,1
69,late_neonatal,680,baseline,0.101275,state_person_time,transition_count,1
70,late_neonatal,829,baseline,0.103557,state_person_time,transition_count,1
71,late_neonatal,946,baseline,0.108025,state_person_time,transition_count,1


In [40]:
vop.describe(data25.mam_duration)

,,,,,count,mean,std,min,2.5%,50%,97.5%,max
age,denominator_measure,multiplier,numerator_measure,scenario,,,,,,,,
1-5_months,transition_count,1,state_person_time,baseline,12.0,0.103723,0.001484,0.101561,0.101710,0.103625,0.105854,0.105860
12_to_23_months,transition_count,1,state_person_time,baseline,12.0,0.111759,0.001229,0.109899,0.110121,0.111545,0.113910,0.114013
2_to_4,transition_count,1,state_person_time,baseline,12.0,0.118076,0.000742,0.116789,0.116871,0.117916,0.119097,0.119172
6-11_months,transition_count,1,state_person_time,baseline,12.0,0.105100,0.001524,0.102181,0.102633,0.104950,0.107967,0.108660
early_neonatal,transition_count,1,state_person_time,baseline,12.0,0.111317,0.001454,0.108580,0.108718,0.111542,0.113528,0.113814
late_neonatal,transition_count,1,state_person_time,baseline,12.0,0.104537,0.002313,0.101275,0.101355,0.104452,0.107877,0.108025
